## Adaptation Prompt Tuning 

In [ ]:
import os
os.environ["WANDB_PROJECT"]="prompt_learning_methods"
from enum import Enum
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed
from peft import get_peft_config, get_peft_model, AdaptionPromptConfig, TaskType, PeftType
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
seed = 42
set_seed(seed)
device = "cuda"
model_name_or_path = "meta-llama/Llama-2-7b-hf"
tokenizer_name_or_path = "meta-llama/Llama-2-7b-hf"

### Dataset Preparation

In [ ]:
from datasets import load_dataset
dataset_name = "jaykin01/advertisement-copy"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
template = """{% for message in messages %}\n{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% if loop.last and add_generation_prompt %}{{'<|im_start|>assistant\n' }}{% endif %}{% endfor %}"""
tokenizer.chat_template = template

system_prompt = """Create a text ad given the following product and description."""

def preprocess(samples):
    batch = []
    for product, desc, ad_copy in zip(samples["product"],samples["description"],samples["ad"]):
        conversation = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"""Product: {product}\nDescription: {desc}\n"""},
            {"role": "assistant", "content": f"""Ad: {ad_copy}\n"""},
        ]
        batch.append(tokenizer.apply_chat_template(conversation, tokenize=False))
    return {"content": batch}

dataset = load_dataset(dataset_name)
dataset = dataset.map(
    preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

dataset["train"][0]

In [ ]:
dataset = dataset["train"].train_test_split(0.1)
dataset

## Create the PEFT model

### Adaptation Prompt Tuning config 

In [ ]:
peft_config = AdaptionPromptConfig(adapter_len=32, 
                                   adapter_layers=30, 
                                   task_type=TaskType.CAUSAL_LM)

In [ ]:
class ChatmlSpecialTokens(str, Enum):
    user = "<|im_start|>user"
    assistant = "<|im_start|>assistant"
    system = "<|im_start|>system"
    eos_token = "<|im_end|>"
    bos_token = "<s>"
    pad_token = "<pad>"

    @classmethod
    def list(cls):
        return [c.value for c in cls]
response_template = "<|im_start|>assistant\n"

tokenizer = AutoTokenizer.from_pretrained(
        model_name_or_path,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        bos_token=ChatmlSpecialTokens.bos_token.value,
        eos_token=ChatmlSpecialTokens.eos_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
        trust_remote_code=True
    )
tokenizer.chat_template = template
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
